In [16]:
import numpy as np

In [40]:
my_data = np.genfromtxt('movie_metadata_copy_Kay_edited.csv', delimiter=',')
print(my_data.shape)



genreList = np.genfromtxt('Movie Genre.csv', delimiter=',')
print(genreList.shape)


(5042, 28)
(26,)


In [34]:
print(my_data[1:5,1:5])

[[    nan    723.    178.      0.]
 [    nan    302.    169.    563.]
 [    nan    602.    148.      0.]
 [    nan    813.    164.  22000.]]


In [32]:
for i in range (0,26):
    print(genreList[i])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [20]:
genresAllMovies = my_data[:,10]


for every movie
    for every genre for that movie
        add my movieID to a list 

genreDict = {}



    numMovies = genresAllMovies.size
    
    for i in range(0,numMovies):
        genreString = numMovies[i]
        genres=genreString.split("|")
        
        
        
        
        
print(numMovies)
    

NameError: name 'numMovies' is not defined

In [35]:
my_data[1,:]

array([             nan,              nan,   7.23000000e+02,
         1.78000000e+02,   0.00000000e+00,   8.55000000e+02,
                    nan,   1.00000000e+03,   7.60505847e+08,
                    nan,              nan,              nan,
         8.86204000e+05,   4.83400000e+03,              nan,
         0.00000000e+00,              nan,              nan,
         3.05400000e+03,              nan,              nan,
                    nan,   2.37000000e+08,   2.00900000e+03,
         9.36000000e+02,   7.90000000e+00,   1.78000000e+00,
         3.30000000e+04])

In [45]:
my_data = np.genfromtxt('movie_metadata_copy_Kay_edited.csv', dtype=None, delimiter=',')
print(my_data.shape)

my_data[2,:]

(5042, 28)


array(['Color', 'Gore Verbinski', '302', '169', '563', '1000',
       'Orlando Bloom', '40000', '309404152', 'Action|Adventure|Fantasy',
       'Johnny Depp', '"""Pirates of the Caribbean: At World\'s End"""',
       '471220', '48350', 'Jack Davenport', '0',
       'goddess|marriage ceremony|marriage proposal|pirate|singapore',
       'http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1', '1238',
       'English', 'USA', 'PG-13', '300000000', '2007', '5000', '7.1',
       '2.35', '0'], 
      dtype='|S149')

In [53]:
example = my_data[7,:]
example['genre']

'sandman|spider man|symbiote|venom|villain'

In [55]:
col_names = my_data[0,:]